In [4]:
from sklearn.neural_network import MLPRegressor
from sklearn.neural_network import MLPClassifier
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
import matplotlib.pyplot as plt
from sklearn.metrics import RocCurveDisplay
import numpy as np
from sklearn import tree

from NN_Classifier import *
from NN_Regressor import *
from DecisonTree import *
from CounterfactualSurrogateModel import *


Import all Models

In [5]:


datasets = [
    {
        "name":"iris",
        "model": NN_Classifier,
        "continous_features": ['sepal_length', 'sepal_width', 'petal_length', 'petal_width'],
        "category_features":[],
        "regression":False,
        "classname": "class"

    },
    {
        "name":"dry-bean",
        "model": NN_Classifier,
        "continous_features": ['Area', 'Perimeter', 'MajorAxisLength', 'MinorAxisLength', 'AspectRation', 'Eccentricity', 'ConvexArea', 'EquivDiameter', 'Extent', 'Solidity', 'roundness', 'Compactness', 'ShapeFactor1', 'ShapeFactor2', 'ShapeFactor3', 'ShapeFactor4'],
        "category_features": [],
        "regression": False,
        "classname": "class"
    },
    {
        "name":"adult",
        "model": NN_Classifier,
        "continous_features": ['age', 'fnlwgt', 'capital-gain', 'capital-loss', 'hours-per-week', 'education-num'],
        "category_features": ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'native-country'],
        "regression": False,
        "classname": "class"
    },
    {
        "name":"bike-sharing-hourly",
        "model": NN_Regressor,
        "continous_features": ['dteday', 'holiday', 'weekday', 'workingday', 'season', 'yr', 'mnth', 'hr', 'temp', 'atemp', 'hum', 'windspeed'],
        "category_features": ['weathersit'],
        "regression": True,
        "classname":"cnt",
    },
    {
        "name":"wine-quality",
        "model": NN_Regressor,
        "continous_features": ["fixed acidity", "volatile acidity", "citric acid", "residual sugar", "is_red", "chlorides", "free sulfur dioxide", "total sulfur dioxide", "density", "pH", "sulphates", "alcohol"],
        "category_features": [],
        "regression": True,
        "classname": "quality"
    },
]


In [6]:
N_SAMPES = 500
experimentName = f"global-{N_SAMPES}"


for dataset in datasets:
    model = dataset['model'](dataset['name'],
        categorical_features=dataset['category_features'],
        continous_features=dataset['continous_features'],
        classname=dataset['classname'],
    )

    model.load_data()
    model.split_data()
    # model.train()
    model.loadModel()
    model.evaluate()

    validationData = model.getGlobalRandomSample(n_samples=N_SAMPES)

    surrogateModel = DecisonTree(dataset['name'],
        categorical_features=dataset['category_features'],
        continous_features=dataset['continous_features'],
        classname=dataset['classname'],
    )

    trainingData = pd.concat(
        [
            pd.read_csv(f"counterfactuals/{experimentName}-{model.name}-1.csv"),
            pd.read_csv(f"counterfactuals/{experimentName}-{model.name}-2.csv")
        ]
    )

    surrogateModel.load_data(data=trainingData)
    surrogateModel.setTrainingData(trainingData)
    surrogateModel.setValidationData(validationData)
    surrogateModel.train()
    surrogateModel.evaluate()

    # surrogateModel = CountefactualSurrogateModel(dataset['name'],
    #     categorical_features=model.categorical_features,
    #     continous_features=model.continous_features,
    #     className=dataset['classname'],
    #     fileModifer=experimentName,
    #     regression=dataset['regression'],
    #     n_samples=1
    # )

    # validationData = model.getGlobalRandomSample(n_samples=N_SAMPES)

    # surrogateModel.loadModel(model.clf)
    # # surrogateModel.loadDataSet(model_data)

    # print("CounterFactual")
    # surrogateModel.generateTree(dataset=[1, 2],
    #     localisedValidationData=validationData
    # )

    # print("Control")
    # surrogateModel.generateTree(
    #     localisedData=model.getGlobalRandomSample(n_samples=N_SAMPES),
    #     localisedValidationData=validationData
    # )

    # end_time = time.time()
    # print("Elapsed time: ", end_time - start_time, "seconds")
    # print(f"SampleSize : {N_SAMPES}")


******** iris nn ********
['Iris-setosa' 'Iris-versicolor' 'Iris-virginica']
AUC score: 1.0
Accuracy: 1.0
['Iris-setosa' 'Iris-versicolor' 'Iris-virginica']
AUC score: 0.9099119249417319
Accuracy: 0.86
Depth of the tree: 10
CounterFactual
Iris-setosa auc: 0.9128937388747
Iris-versicolor auc: 0.8306236097998111
Iris-virginica auc: 0.8845752996352266
AUC score: 0.8760308827699127
Accuracy: 0.83
Depth of the tree: 10
Control
Iris-setosa auc: 0.9604906740964322
Iris-versicolor auc: 0.9039861740090573
Iris-virginica auc: 0.8995508573413732
AUC score: 0.9213425684822876
Accuracy: 0.904
Depth of the tree: 10


NameError: name 'time' is not defined